# Test translation

In [1]:
import ctypes
import cv2
import numpy as np
import time

In [2]:
ekmi = ctypes.cdll.LoadLibrary('./ekmi.so')
_get_ekmi = ekmi.get_ekmi
_get_ekmi.argtypes = [
    ctypes.c_int, 
    np.ctypeslib.ndpointer(dtype=np.uintp, flags="C", ndim=1), 
    ctypes.c_int,
    ctypes.c_double
]
_get_ekmi.restype = ctypes.POINTER(ctypes.POINTER(ctypes.c_double))


def get_ekmi(order, img, p):
    
    imgpp = (img.__array_interface__['data'][0] + np.arange(img.shape[0])* img.strides[0]).astype(np.uintp) 
    _res = _get_ekmi(order, imgpp, img.shape[0], p)

    # copy array
    res = np.zeros((order, order))
    for i in range(order):
        for j in range(order):
            res[i, j] = _res[i][j]
    ekmi.free_ekmi_rst() # free memory
    return res

In [3]:
def ekmi_from_fname(fname):
    p = 0.5
    N = 16
    tik = time.time()
    order = 6

    img = (cv2.imread(fname)[:, :, 0] == 255).astype('int32')
    retval = get_ekmi(order, img, p)
    tok = time.time()
    print(tok - tik)
    return retval

k1 = ekmi_from_fname("/Users/kx/desktop/test/1_small.png")
k2 = ekmi_from_fname("/Users/kx/desktop/test/1_small_t.png")
print(k1)
print(k2)
sum(sum((k2 - k1) ** 2))

0.0019941329956054688
0.0020530223846435547
[[ 5.30000000e+01  2.73333333e+00 -2.95238095e-01 -7.69230769e-02
  -1.98534799e-01 -1.04895105e-01]
 [-7.40000000e+00 -4.04444444e-01 -2.50793651e-01  1.66593407e-01
   8.97191697e-02 -7.88322788e-02]
 [ 1.06380952e+01 -2.53333333e-01 -9.78684807e-02  2.75667190e-02
  -1.88034188e-02 -8.99100899e-03]
 [-2.82857143e+00 -1.03589744e-01 -7.03924647e-02  4.54872600e-02
   2.73928270e-02 -2.20431583e-02]
 [ 2.40952381e+00 -1.79877900e-01 -9.29565672e-02  9.63651733e-03
   1.65932724e-02  1.30589801e-03]
 [-2.22111222e-01  8.01420801e-03  2.80893709e-02  8.55188767e-03
  -6.45191195e-03 -5.34874161e-03]]
[[ 5.30000000e+01 -1.84666667e+01  6.44761905e+00 -2.47252747e+00
   9.67765568e-01 -2.80053280e-01]
 [-7.40000000e+00  2.55555556e+00 -1.17269841e+00  8.91868132e-01
  -7.79633700e-01  5.82062382e-01]
 [ 1.06380952e+01 -4.50857143e+00  1.94294785e+00 -8.75709053e-01
   4.10709925e-01 -1.97342340e-01]
 [-2.82857143e+00  1.02783883e+00 -4.66499215e

539.6029731295722

___

# Test recognition

In [4]:
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier
from ikmi import *

In [19]:
order = 10
p = 0.5

In [20]:
data_dir = '/Users/kx/Docs/github/git_cccr_sl/chinese-calligraphy-recognition-sl/data/shufadict/clean'
chars = [x for x in os.listdir(data_dir) if x[0] != '.']
N = 64
nclass = 10
sample_per_class = 10

In [21]:
class_lookup = [None] * nclass
data = np.zeros((nclass * sample_per_class, N, N), dtype='int32')
label = np.zeros((nclass * sample_per_class), dtype='int')
feature_ekmi = np.zeros((nclass * sample_per_class, order, order))
feature_ikmi = np.zeros((nclass * sample_per_class, order, order))

In [22]:
for i, char in enumerate(chars[:nclass]):
    class_lookup[i] = char
    fnames = glob.glob('{}/{}/*.png'.format(data_dir, char))[:sample_per_class]
    for j, fname in enumerate(fnames):
        img = cv2.imread(fname)
        img = cv2.resize(img, (N, N))
        img = (img[:, :, 0] > 0).astype('int32')
        data[i * nclass + j, :, :] = img
        label[i * nclass + j] = i

In [23]:
for i in tqdm(range(nclass * sample_per_class)):
    img = data[i, :, :]
    feature_ekmi[i, :, :] = get_ekmi(order, img, p)
    feature_ikmi[i, :, :] = wkrchkmoment_single(img)[0][:order, :order]

  0%|          | 0/100 [00:00<?, ?it/s]

In [24]:
# todo: choose feature
feature = feature_ekmi
# feature = feature_ikmi

In [25]:
same = sum(sum((feature[3] - feature[5]) ** 2))
diff = sum(sum((feature[1] - feature[22]) ** 2))
same, diff, diff / same

(32175.572023013174, 17839.829326624033, 0.554452592602373)

In [26]:
# (feature[1,:,:] * 100).round(2)

In [27]:
# (feature[30,:,:] * 100).round(2)

In [28]:
test_count = 2

test_idxs = np.concatenate([[i * nclass + j for i in range(nclass)] for j in range(test_count)])
test_idxs.sort()
train_idxs = [x for x in range(feature.shape[0]) if x not in test_idxs]

test = feature[test_idxs]
test_label = label[test_idxs]
test_feature = feature[test_idxs]

train = feature[train_idxs]
train_label = label[train_idxs]
train_feature = feature[train_idxs]

In [29]:
X = train.reshape((train.shape[0], train.shape[1] * train.shape[2]))
y = test.reshape((test.shape[0], test.shape[1] * test.shape[2]))

In [30]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, train_label)
sum(knn.predict(y) == test_label) / len(test_label)

0.25

In [74]:
# dist = np.zeros((train.shape[0], train.shape[0]))
# for i in range(train.shape[0]):
#     for j in range(train.shape[0]):
#         dist[i, j] = sum(sum((feature[i, :, :] - feature[j, :, :]) ** 2))
        
# np.where((dist > 0) & (dist < 1))